In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
print("current working director: ", os.getcwd())

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from supermarq.benchmarks.hamiltonian_simulation import HamiltonianSimulation
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
import qiskit_aer.noise as noise
from qiskit_addon_utils.slicing import slice_by_depth, combine_slices
from qiskit.quantum_info import hellinger_fidelity
from qiskit import QuantumCircuit
from utils.pce_vs_zne_utils import (
      extrapolate_checks,
      mitigate_zne,
      transpile_circuit,
      run_sampler_job,
      filter_counts,
      get_line_initial_layout
  )
from utils.hw_exp_utils import (pauli_z_expectation, generate_rand_mirror_cliff, ghz_mirror_circ,
                                aggregate_results, save_aggregated_results, save_detailed_results,
                                load_detailed_results, plot_comparison_with_errors, 
                                plot_pcs_extrapolation, create_pcs_circuit, get_mapomatic_payload_scores, find_pcs_ancillas,
                                append_circuit_result, load_incremental_results)
import mthree
from mitiq import zne

In [2]:
def get_ideal_score(circ, shots):
    ideal_sim = AerSimulator()
    ideal_circuit = circ.copy()
    ideal_circuit.measure_all()
    ideal_job = ideal_sim.run(ideal_circuit, shots=shots)
    ideal_counts = ideal_job.result().get_counts()
    ideal_score = pauli_z_expectation(ideal_counts)

    return ideal_score

def get_baseline_score(circ: QuantumCircuit, backend, shots: int, init_layout, opt_level: int = 0):
    baseline_circuit = circ.copy()
    baseline_circuit.measure_all()
    baseline_isa = transpile_circuit(baseline_circuit, backend, layout=init_layout, optimization_level=opt_level)
    baseline_counts = run_sampler_job(baseline_isa, backend, shots=shots, enable_error_mitigation=False)
    baseline_score = pauli_z_expectation(baseline_counts)

    return baseline_score

def get_raw_pcs_counts(circ: QuantumCircuit, backend, shots: int, init_layout, enable_error_mitigation: bool, enable_mthree: bool, opt_level: int = 0):
    
    pcs_circuit_measure = circ.copy()
    pcs_circuit_measure.measure_all()
    pcs_isa = transpile_circuit(pcs_circuit_measure, backend,
                                layout=init_layout, 
                                optimization_level=opt_level)
    raw_pcs_counts = run_sampler_job(pcs_isa, backend, shots, enable_error_mitigation=enable_error_mitigation)

    return raw_pcs_counts

In [ ]:
# Configuration
USE_REAL_HARDWARE = False
BACKEND_NAME = "ibm_kingston"

num_qubits = 4
max_num_checks = 3
circ_depth = 10
num_circuits = 1  # Number of random circuits to average over

pcs_shots = 50_000
shots = pcs_shots * max_num_checks
# shots = 100_000 # total shot budget (also number of shots used for baseline circuit)
pcs_shots = shots // max_num_checks
print(f"total shot budget = {shots}")
print(f"shots per pcs circuit = {pcs_shots}")
# baseline_shots = shots

optimization_level = 0 # temporary parameter?
enable_mthree = True

if max_num_checks == 2:
    pce_methods_list = ["linear"]
else:
    pce_methods_list = ["linear", "exponential"]

only_Z_checks = True
barriers = True

zne_methods_list = ["linear", "richardson"]
scale_factors_list  = [
    [1, 3, 5]
]

fold_methods_list = [zne.scaling.fold_global]

In [ ]:
from qiskit_ibm_runtime.fake_provider import *

# Get noisy backend
service = QiskitRuntimeService() # setup service
real_backend = service.backend(BACKEND_NAME)

if USE_REAL_HARDWARE and QiskitRuntimeService.saved_accounts():
    # Real hardware
    noisy_backend = real_backend
    print(f"Using real hardware: {noisy_backend.name}")
    print(f"basis gate set: {noisy_backend.configuration().basis_gates}")
else:
    # Fake backend with noise model
    # noise_model = noise.NoiseModel.from_backend(real_backend)
    # basis_gates = real_backend.configuration().basis_gates
    # coupling_map = real_backend.configuration().coupling_map
    # noisy_backend = AerSimulator(noise_model=noise_model, basis_gates=basis_gates, coupling_map=coupling_map)
    # print(f"Using fake backend with noise from: {BACKEND_NAME}")
    noisy_backend = FakeFez()
    print(noisy_backend)

In [ ]:
# Generate mirrored random clifford cicuits
rand_circs = [generate_rand_mirror_cliff(num_qubits, circ_depth) for _ in range(num_circuits)]
print(f"Generated {len(rand_circs)} circuits")

In [ ]:
def build_pcs_configs(circuit, payload_layout, backend, max_num_checks, barriers=True):
    """Build PCS configurations for a given payload layout. Returns None if connectivity requirements aren't met."""
    pcs_configs = []
    
    for cid in range(1, max_num_checks + 1):
        ancilla_positions, check_indices, all_connected = find_pcs_ancillas(
            payload_layout, backend, cid,
            verbose=False,
            require_connectivity=True  # Only accept fully connected ancillas
        )
        
        # Check if we have enough connected ancillas
        if len(ancilla_positions) < cid or not all_connected:
            return None  # This layout doesn't have sufficient connectivity
        
        pcs_layout = payload_layout + ancilla_positions
        pcs_circuit = create_pcs_circuit(circuit, check_indices, check_type='Z', barriers=barriers)
        
        pcs_configs.append({
            'num_checks': cid,
            'layout': pcs_layout,
            'check_indices': check_indices,
            'circuit': pcs_circuit,
            'all_connected': all_connected
        })
    
    return pcs_configs

# Main layout determination code - simplified
print("\nDetermining layouts for all circuits...")
circuit_configs = []
qubits_used = set()

for circ_idx, rand_circ in enumerate(rand_circs):
    print(f"\nCircuit {circ_idx + 1}/{num_circuits}:")
    
    # Get candidate layouts from Mapomatic
    payload_scores = get_mapomatic_payload_scores(rand_circ, real_backend, verbose=True)
    print(f"  Found {len(payload_scores)} candidate layouts")
    
    best_payload_layout = None
    successful_pcs_configs = None
    
    # Try layouts in order of quality until one works
    for layout_idx, (candidate_layout, score) in enumerate(payload_scores):
        pcs_configs = build_pcs_configs(
            rand_circ, candidate_layout, real_backend, 
            max_num_checks, barriers
        )
        
        if pcs_configs is not None:
            # This layout works!
            best_payload_layout = candidate_layout
            successful_pcs_configs = pcs_configs
            print(f"  ✅ Layout #{layout_idx + 1} selected: {candidate_layout} (score: {score:.4f})")
            break
    
    # If no layout has full connectivity, fail
    if best_payload_layout is None:
        raise RuntimeError(
            f"❌ Circuit {circ_idx + 1}: No layout found with sufficient ancilla connectivity!\n"
            f"   Tried {len(payload_scores)} layouts but none had {max_num_checks} connected ancillas.\n"
            f"   Consider reducing max_num_checks or using a different backend."
        )
    
    # Track all qubits used
    qubits_used.update(best_payload_layout)
    for config in successful_pcs_configs:
        qubits_used.update(config['layout'])
    
    # Store configuration
    circuit_configs.append({
        'circuit': rand_circ,
        'payload_layout': best_payload_layout,
        'pcs_configs': successful_pcs_configs
    })
    
    # Print summary
    conn_status = ["✅" if c['all_connected'] else "⚠️" for c in successful_pcs_configs]
    print(f"    Connectivity: {' '.join(conn_status)} for {len(successful_pcs_configs)} check(s)")

print(f"\n✅ Successfully found layouts for all {num_circuits} circuits")
print(f"Total qubits used: {len(qubits_used)} - {sorted(qubits_used)}")

In [ ]:
circuit_configs[0]["payload_layout"]

In [9]:
# Calibrate mthree 
if enable_mthree and USE_REAL_HARDWARE:
    print("M3 enabled")
    print("Calibrating...")

    mit = mthree.M3Mitigation(real_backend) # Specify a mitigator object targeting a given backend

    # Compute the 1Q calibration matrices for the given qubits and given number of shots
    # By default it is over all backend qubits at 10000 shots.
    mit.cals_from_system(list(qubits_used))

    print("Done.")

In [ ]:
# Run experiments over multiple circuits
all_results = []

for circ_idx in range(num_circuits):
    print(f"\n{'='*60}")
    print(f"Circuit {circ_idx + 1}/{num_circuits}")
    print(f"{'='*60}")

    qc = rand_circs[circ_idx]
    payload_layout = circuit_configs[circ_idx]["payload_layout"]
    payload_layout = None
    print(f"payload layout for circ #{circ_idx+1}: {payload_layout}")

    ideal_score = get_ideal_score(qc, shots)
    print(f"ideal expectation: {ideal_score}")

    # Get baseline score using Mapomatic layout
    baseline_score = get_baseline_score(qc, noisy_backend, shots=shots, init_layout=payload_layout, opt_level=optimization_level)

    print(f"baseline score: {baseline_score}")

    # Apply PCS protection using Mapomatic layouts + error rate prioritized ancillas
    pcs_scores = []
    pcs_measured_values = []  # Store detailed measurement data
    
    for cid in range(1, max_num_checks + 1):
        print(f"\n--- PCS with {cid} checks ---")

        print(f"number of shots per pcs circuit = {pcs_shots}")

        # Retrieve pre-computed PCS configuration
        pcs_config = circuit_configs[circ_idx]["pcs_configs"][cid - 1]  # cid-1 because list is 0-indexed

        pcs_circuit = pcs_config['circuit']
        pcs_layout = pcs_config['layout']
        check_indices = pcs_config['check_indices']

        print("resulting logical pcs circuit:")
        display(pcs_circuit.draw("mpl", fold=-1))
        print(f"PCS layout: {pcs_layout}")
        print(f"Check indices: {check_indices}")

        raw_pcs_counts = get_raw_pcs_counts(pcs_circuit, noisy_backend, shots=pcs_shots, init_layout=pcs_layout,
                                            enable_error_mitigation=True, enable_mthree=enable_mthree)

        if enable_mthree: 
            prefiltered_pcs_counts = mit.apply_correction(raw_pcs_counts, pcs_layout)
        else:
            prefiltered_pcs_counts = raw_pcs_counts 

        signs = ["+1"] * cid  # each check will have a positive sign for mirror circuits
        filtered_counts = filter_counts(cid, signs, prefiltered_pcs_counts)
        total_filtered = sum(filtered_counts.values())
        print("number of filtered counts: ", total_filtered, "with ", cid, "checks")

        pcs_score = pauli_z_expectation(filtered_counts)
        pcs_scores.append(pcs_score)
        
        # Store detailed measurement data
        pcs_measurement_data = {
            'num_checks': cid,
            'signs': signs,
            'raw_counts': raw_pcs_counts,
            'corrected_counts': prefiltered_pcs_counts,
            'filtered_counts': filtered_counts,
            'total_raw_shots': sum(raw_pcs_counts.values()),
            'total_corrected_shots': sum(prefiltered_pcs_counts.values()),
            'total_filtered_shots': total_filtered,
            'post_selection_rate': total_filtered / sum(prefiltered_pcs_counts.values()) if sum(prefiltered_pcs_counts.values()) > 0 else 0,
            'expectation_value': pcs_score,
            'check_indices': check_indices,
            'layout': pcs_layout
        }
        pcs_measured_values.append(pcs_measurement_data)
        
        print(f"   PCS score: {pcs_score:.6f}")

    # PCE extrapolation
    pce_results = {}
    extrap_targets = range(max_num_checks + 1, num_qubits + 1)
    for num_fit_points in range(2, max_num_checks + 1):
        for method in pce_methods_list:
            try:
                extrap_values, _ = extrapolate_checks(
                    num_checks_to_fit=num_fit_points,
                    extrap_checks=extrap_targets,
                    expectation_values=pcs_scores,
                    method=method,
                    show_plot=True
                )
                key = f"pce_{method}_{num_fit_points}pts"
                pce_results[key] = extrap_values[0]
            except Exception as e:
                print(f"  PCE {method} failed: {e}")
                pce_results[f"pce_{method}_{num_fit_points}pts"] = np.nan

    # ZNE - using same Mapomatic payload layout as baseline
    zne_results = []
    for method, scales, fold in itertools.product(zne_methods_list, scale_factors_list, fold_methods_list):
        if method == "richardson":
          if any(not isinstance(scale, int) and not scale.is_integer() for scale in scales):
              print(f"Skipping Richardson with non-integer scales: {scales}")
              continue
          
        observable = "Z"*num_qubits
        zne_shots = shots // len(scales)
        print(f"number of shots per zne circuit = {zne_shots}")

        zne_score = mitigate_zne(
            qc, 
            noisy_backend,
            pauli_string=observable, 
            shots=zne_shots,
            method=method,
            scale_factors=scales,
            fold_method=fold,
            layout=payload_layout,  # Use same Mapomatic layout
            enable_error_mitigation=True
        )
        zne_results.append({
            "method": method,
            "scales": scales,
            "fold": fold.__name__,
            "score": zne_score
        })
        
    # Store results for this circuit
    circuit_results = {
        'circuit_idx': circ_idx,
        'ideal': ideal_score,
        'baseline': baseline_score,
        'pcs': pcs_scores,
        'pcs_measured': pcs_measured_values,  # Add detailed measurement data
        'pce': pce_results,
        'zne': zne_results
    }
    all_results.append(circuit_results)
    
    print(f"Circuit {circ_idx + 1} complete:")
    print(f"  Ideal: {ideal_score:.4f}")
    print(f"  Baseline: {baseline_score:.4f}")
    print(f"  Best PCS: {max(pcs_scores):.4f}")
    valid_pce = [v for v in pce_results.values() if not np.isnan(v)]
    if valid_pce:
        print(f"  Best PCE: {max(valid_pce):.4f}")
    if zne_results:
        print(f"  Best ZNE: {max([r['score'] for r in zne_results]):.4f}")

print(f"\n{'='*60}")
print("🎉 All circuits complete with Mapomatic layouts!")
print(f"{'='*60}")

In [ ]:
aggregated_results = aggregate_results(all_results, max_num_checks)

print("\n=== AGGREGATED RESULTS ===")
for method, stats in aggregated_results.items():
    print(f"{method}: {stats['mean']:.4f} ± {stats['std']:.4f}")

In [ ]:
df = save_aggregated_results(aggregated_results, num_qubits, circ_depth, num_circuits, BACKEND_NAME, USE_REAL_HARDWARE)

In [ ]:
# Save detailed results with all individual data points
detailed_filepath = save_detailed_results(
    all_results, 
    num_qubits=num_qubits, 
    circ_depth=circ_depth,
    backend_name=BACKEND_NAME,
    use_real_hardware=USE_REAL_HARDWARE,
    max_num_checks=max_num_checks
)

print(f"\nDetailed results saved!")
print(f"You can load them later using: load_detailed_results('{detailed_filepath}')")

In [ ]:
plot_comparison_with_errors(df, num_qubits, circ_depth, num_circuits, backend_name=BACKEND_NAME)

In [ ]:
plot_pcs_extrapolation(df, num_qubits, circ_depth, backend_name=BACKEND_NAME)

In [ ]:
# Example: Load and analyze detailed results
# Uncomment to use:

loaded_data = load_detailed_results(detailed_filepath)

# Access metadata
print("Metadata:")
for key, value in loaded_data['metadata'].items():
    print(f"  {key}: {value}")

# Analyze individual circuit results
print(f"\n{loaded_data['metadata']['num_circuits']} circuits analyzed:")
for circuit in loaded_data['circuits']:
    print(f"\nCircuit {circuit['circuit_idx'] + 1}:")
    print(f"  Ideal: {circuit['ideal_score']:.4f}")
    print(f"  Baseline: {circuit['baseline_score']:.4f}")
    print(f"  PCS scores: {[f'{s:.4f}' for s in circuit['pcs_scores']]}")
    
    # Show PCE extrapolations
    for pce_key, pce_data in circuit['pce_extrapolations'].items():
        print(f"  {pce_key}: {pce_data['extrapolated_value']:.4f} "
              f"(using {pce_data['fit_points']} points)")
    
    # Show ZNE results if available
    for zne_result in circuit['zne_results']:
        print(f"  ZNE {zne_result['method']}: {zne_result['extrapolated_value']:.4f}")